In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)
    
!nvidia-smi

2023-07-09 20:41:52.847077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 20:41:53.144872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Sun Jul  9 20:41:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Graphics...  On   | 00000000:01:00.0 Off |                  Off |
|  0%   39C    P8    43W / 450W |      8MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
train_images = train_images/255
test_images = test_images/255

In [4]:
model = keras.Sequential([
    keras.layers.Conv2D(64,(3,3), activation = 'relu', input_shape=(28,28,1)),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPool2D(2, 2), 

    # old layers
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

2023-07-09 20:46:14.626890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1024 MB memory:  -> device: 0, name: NVIDIA Graphics Device, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 256)               4

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        """
        Halts training after reaching 60 precent accuracy
        Args:
            epochs (integer) - index of epoch (required but unused in the function definition below)
            logs (dict) - metric results from the training epock
        """
        
        # Check accuracy 
        if (logs.get('accuracy') > 0.98):
            
            #Stop if threshold is met
            print('\nAccuracy is higher than 98%, cancelling training.')
            self.model.stop_training = True
            
#instantiate the class
callbacks = myCallback()

In [13]:
model.fit(train_images, train_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 2s 910us/step - loss: 0.0872 - accuracy: 0.9670
Epoch 2/10
1875/1875 [==============================] - 2s 865us/step - loss: 0.0764 - accuracy: 0.9707
Epoch 3/10
1875/1875 [==============================] - 2s 887us/step - loss: 0.0669 - accuracy: 0.9746
Epoch 4/10
1875/1875 [==============================] - 2s 900us/step - loss: 0.0594 - accuracy: 0.9781
Epoch 5/10
1875/1875 [==============================] - 2s 941us/step - loss: 0.0577 - accuracy: 0.9779
Epoch 6/10
1847/1875 [============================>.] - ETA: 0s - loss: 0.0528 - accuracy: 0.9803
Accuracy is higher than 98%, cancelling training.
1875/1875 [==============================] - 2s 928us/step - loss: 0.0529 - accuracy: 0.9804


In [14]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 651us/step - loss: 0.4352 - accuracy: 0.9113


[0.4351727068424225, 0.911300003528595]